## Settings

In [34]:
# import libraries
from sklearn.feature_selection import VarianceThreshold, SelectKBest, chi2
from statsmodels.stats.outliers_influence import variance_inflation_factor
import feature_engineering as fe
import pandas as pd
import numpy as np

In [35]:
# load data
train_data = pd.read_csv("../data/train_data.csv")

In [36]:
# preprocess data
pd.set_option('display.max_columns', None)
original_data = fe.feature_engineering(train_data)
original_data

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,userID_answerCode_mean,userID_answerCode_count,userID_answerCode_sum,userID_answerCode_var,userID_answerCode_median,testId_answerCode_mean,testId_answerCode_count,testId_answerCode_sum,testId_answerCode_var,testId_answerCode_median,assessmentItemID_answerCode_mean,assessmentItemID_answerCode_count,assessmentItemID_answerCode_sum,assessmentItemID_answerCode_var,assessmentItemID_answerCode_median,KnowledgeTag_answerCode_mean,KnowledgeTag_answerCode_count,KnowledgeTag_answerCode_sum,KnowledgeTag_answerCode_var,KnowledgeTag_answerCode_median,year,month,day,hour,minute,second,dayofweek,dayofweek_answerCode_mean,dayofweek_answerCode_count,dayofweek_answerCode_sum,dayofweek_answerCode_var,dayofweek_answerCode_median,first3,mid3,last3,userID_first3_answerCode_mean,userID_first3_answerCode_count,userID_first3_answerCode_sum,userID_first3_answerCode_var,userID_first3_answerCode_median,hour_answerCode_mean,hour_answerCode_count,hour_answerCode_sum,hour_answerCode_var,hour_answerCode_median,hour_answerCode_Level,month_answerCode_mean,month_answerCode_count,month_answerCode_sum,month_answerCode_var,month_answerCode_median,elapsedTime,userID_elapsedTime_median,KnowledgeTag_elapsedTime_median,assessmentItemID_elapsedTime_median,testId_elapsedTime_median,userID_answerCode_elapsedTime_median,KnowledgeTag_answerCode_elapsedTime_median,assessmentItemID_answerCode_elapsedTime_median,testId_answerCode_elapsedTime_median
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,0.630872,745,470,0.233185,1.0,0.947683,1338,1268,0.049617,1.0,0.982063,223,219,0.017695,1.0,0.955022,667,637,0.043019,1.0,2020,3,24,0,17,11,1,0.667046,458988,306166,0.222096,1.0,060,001,001,0.791908,346,274,0.165268,1.0,0.649446,101137,65683,0.227668,1.0,1,0.682643,174482,119109,0.216643,1.0,45.520031,27.000000,45.520031,45.520031,15.000000,31.500000,45.520031,45.520031,15.000000
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,0.630872,745,470,0.233185,1.0,0.947683,1338,1268,0.049617,1.0,0.964126,223,215,0.034743,1.0,0.913187,3329,3040,0.079300,1.0,2020,3,24,0,17,14,1,0.667046,458988,306166,0.222096,1.0,060,001,002,0.791908,346,274,0.165268,1.0,0.649446,101137,65683,0.227668,1.0,1,0.682643,174482,119109,0.216643,1.0,3.000000,27.000000,15.000000,6.000000,15.000000,31.500000,14.000000,6.000000,15.000000
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,0.630872,745,470,0.233185,1.0,0.947683,1338,1268,0.049617,1.0,0.910314,223,203,0.082010,1.0,0.913187,3329,3040,0.079300,1.0,2020,3,24,0,17,22,1,0.667046,458988,306166,0.222096,1.0,060,001,003,0.791908,346,274,0.165268,1.0,0.649446,101137,65683,0.227668,1.0,1,0.682643,174482,119109,0.216643,1.0,8.000000,27.000000,15.000000,14.000000,15.000000,31.500000,14.000000,14.000000,15.000000
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,0.630872,745,470,0.233185,1.0,0.947683,1338,1268,0.049617,1.0,0.968610,223,216,0.030542,1.0,0.913187,3329,3040,0.079300,1.0,2020,3,24,0,17,29,1,0.667046,458988,306166,0.222096,1.0,060,001,004,0.791908,346,274,0.165268,1.0,0.649446,101137,65683,0.227668,1.0,1,0.682643,174482,119109,0.216643,1.0,7.000000,27.000000,15.000000,10.000000,15.000000,31.500000,14.000000,10.000000,15.000000
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,0.630872,745,470,0.233185,1.0,0.947683,1338,1268,0.049617,1.0,0.941704,223,210,0.055145,1.0,0.913187,3329,3040,0.079300,1.0,2020,3,24,0,17,36,1,0.667046,458988,306166,0.222096,1.0,060,001,005,0.791908,346,274,0.165268,1.0,0.649446,101137,65683,0.227668,1.0,1,0.682643,174482,119109,0.216643,1.0,7.000000,27.000000,15.000000,11.000000,15.000000,31.500000,14.000000,11.000000,15.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,

In [37]:
# get continuous data
continuous_data = original_data.drop(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp', 'KnowledgeTag', 'year', 'day', 'minute','second', 'first3', 'mid3', 'last3', 'hour_answerCode_Level'], axis=1)

* 범주형 변수와 유의미한 영향을 주지 못할 것으로 판단되는 변수는 제외
* 연속형 변수만을 사용하여 평가함
* 범주형 변수: userID, assessmentID, testId, answerCode, KnowledgeTag, first3, hour_answerCode_Level
* 무의미한 변수: Timestamp, year, day, minute, second, mid3, last3
* 변수의 유의미성에 대한 평가는 매우 주관적임

## 1. 분산 임계 (Variance Threshold)
* 분산값이 작은(변별력이 낮은) 변수를 제외하는 작업

In [38]:
# set features: input 52 continuous features
X = continuous_data.copy()

selector = VarianceThreshold(threshold=2)
selector.fit_transform(X).shape

(2266586, 28)

* 분산 2 기준,
* 52개의 연속형 변수 중, 28개의 feature가 선택되었고, 24개의 feature가 제외되었다

In [39]:
# 선택된 feature의 정보
X.columns[selector.get_support()]

Index(['userID_answerCode_count', 'userID_answerCode_sum',
       'testId_answerCode_count', 'testId_answerCode_sum',
       'assessmentItemID_answerCode_count', 'assessmentItemID_answerCode_sum',
       'KnowledgeTag_answerCode_count', 'KnowledgeTag_answerCode_sum', 'month',
       'hour', 'dayofweek', 'dayofweek_answerCode_count',
       'dayofweek_answerCode_sum', 'userID_first3_answerCode_count',
       'userID_first3_answerCode_sum', 'hour_answerCode_count',
       'hour_answerCode_sum', 'month_answerCode_count', 'month_answerCode_sum',
       'elapsedTime', 'userID_elapsedTime_median',
       'KnowledgeTag_elapsedTime_median',
       'assessmentItemID_elapsedTime_median', 'testId_elapsedTime_median',
       'userID_answerCode_elapsedTime_median',
       'KnowledgeTag_answerCode_elapsedTime_median',
       'assessmentItemID_answerCode_elapsedTime_median',
       'testId_answerCode_elapsedTime_median'],
      dtype='object')

In [40]:
selector.variances_[selector.get_support()]

array([1.35153579e+05, 7.72494512e+04, 3.24859959e+05, 1.00529904e+05,
       3.66676928e+03, 3.74589639e+03, 3.34062791e+06, 2.06978388e+06,
       7.05479413e+00, 3.53075493e+01, 2.90449227e+00, 9.58095967e+09,
       4.31588169e+09, 5.29997855e+04, 3.21572317e+04, 1.65330647e+09,
       7.06075093e+08, 7.70345533e+09, 3.60435271e+09, 2.40268895e+03,
       1.48881623e+02, 1.64255843e+02, 2.99703725e+02, 1.68784371e+02,
       2.04136439e+02, 1.91150321e+02, 3.60705793e+02, 2.01300691e+02])

## 2. 카이제곱 독립검정 (Chi-squared Test)
* output(answerCode)과의 상관관계가 높은 input feature들을 찾는 작업

In [41]:
# set features: input 28 features
X = continuous_data.copy()
X = X[['userID_answerCode_count', 'userID_answerCode_sum',
       'testId_answerCode_count', 'testId_answerCode_sum',
       'assessmentItemID_answerCode_count', 'assessmentItemID_answerCode_sum',
       'KnowledgeTag_answerCode_count', 'KnowledgeTag_answerCode_sum', 'month',
       'hour', 'dayofweek', 'dayofweek_answerCode_count',
       'dayofweek_answerCode_sum', 'userID_first3_answerCode_count',
       'userID_first3_answerCode_sum', 'hour_answerCode_count',
       'hour_answerCode_sum', 'month_answerCode_count', 'month_answerCode_sum',
       'elapsedTime', 'userID_elapsedTime_median',
       'KnowledgeTag_elapsedTime_median',
       'assessmentItemID_elapsedTime_median', 'testId_elapsedTime_median',
       'userID_answerCode_elapsedTime_median',
       'KnowledgeTag_answerCode_elapsedTime_median',
       'assessmentItemID_answerCode_elapsedTime_median',
       'testId_answerCode_elapsedTime_median']]
X

,userID_answerCode_count,userID_answerCode_sum,testId_answerCode_count,testId_answerCode_sum,assessmentItemID_answerCode_count,assessmentItemID_answerCode_sum,KnowledgeTag_answerCode_count,KnowledgeTag_answerCode_sum,month,hour,dayofweek,dayofweek_answerCode_count,dayofweek_answerCode_sum,userID_first3_answerCode_count,userID_first3_answerCode_sum,hour_answerCode_count,hour_answerCode_sum,month_answerCode_count,month_answerCode_sum,elapsedTime,userID_elapsedTime_median,KnowledgeTag_elapsedTime_median,assessmentItemID_elapsedTime_median,testId_elapsedTime_median,userID_answerCode_elapsedTime_median,KnowledgeTag_answerCode_elapsedTime_median,assessmentItemID_answerCode_elapsedTime_median,testId_answerCode_elapsedTime_median
0,745,470,1338,1268,223,219,667,637,3,0,1,458988,306166,346,274,101137,65683,174482,119109,45.520031,27.000000,45.520031,45.520031,15.000000,31.500000,45.520031,45.520031,15.000000
1,745,470,1338,1268,223,215,3329,3040,3,0,1,458988,306166,346,274,101137,65683,174482,119109,3.000000,27.000000,15.000000,6.000000,15.000000,31.500000,14.000000,6.000000,15.000000
2,745,470,1338,1268,223,203,3329,3040,3,0,1,458988,306166,346,274,101137,65683,174482,119109,8.000000,27.000000,15.000000,14.000000,15.000000,31.500000,14.000000,14.000000,15.000000
3,745,470,1338,1268,223,216,3329,3040,3,0,1,458988,306166,346,274,101137,65683,174482,119109,7.000000,27.000000,15.000000,10.000000,15.000000,31.500000,14.000000,10.000000,15.000000
4,745,470,1338,1268,223,210,3329,3040,3,0,1,458988,306166,346,274,101137,65683,174482,119109,7.000000,27.000000,15.000000,11.000000,15.000000,31.500000,14.000000,11.000000,15.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,9,5,1390,921,278,121,4080,2814,6,6,4,387283,249163,5,1,167792,105699,227481,148310,24.000000,45.520031,42.000000,34.500000,42.000000,39.760015,40.000000,27.000000,45.000000
2266582,9,5,1096,718,274,178,3151,2199,8,1,4,387283,249163,4,4,136702,88080,316115,207877,45.520031,45.520031,31.000000,45.520031,45.520031,45.520031,29.000000,45.520031,45.520031
2266583,9,5,1096,718,274,174,3151,2199,8,1,4,387283,249163,4,4,136702,88080,316115,207877,11.000000,45.520031,31.000000,17.000000,45.520031,45.520031,29.000000,16.000000,45.520031
2266584,9,5,1096,718,274,217,3151,2199,8,1,4,387283,249163,4,4,136702,88080,316115,207877,46.000000,45.520031,31.000000,14.000000,45.520031,45.520031,29.000000,14.000000,45.520031


In [42]:
# scaling data in 0 to 1
for col in X.columns:
    X[col] = X[col] / X[col].max()
X

,userID_answerCode_count,userID_answerCode_sum,testId_answerCode_count,testId_answerCode_sum,assessmentItemID_answerCode_count,assessmentItemID_answerCode_sum,KnowledgeTag_answerCode_count,KnowledgeTag_answerCode_sum,month,hour,dayofweek,dayofweek_answerCode_count,dayofweek_answerCode_sum,userID_first3_answerCode_count,userID_first3_answerCode_sum,hour_answerCode_count,hour_answerCode_sum,month_answerCode_count,month_answerCode_sum,elapsedTime,userID_elapsedTime_median,KnowledgeTag_elapsedTime_median,assessmentItemID_elapsedTime_median,testId_elapsedTime_median,userID_answerCode_elapsedTime_median,KnowledgeTag_answerCode_elapsedTime_median,assessmentItemID_answerCode_elapsedTime_median,testId_answerCode_elapsedTime_median
0,0.400538,0.302835,0.337879,0.474906,0.491189,0.693038,0.051738,0.059706,0.250000,0.000000,0.166667,1.000000,1.000000,0.272656,0.252302,0.602752,0.613854,0.514294,0.524883,0.151733,0.350649,0.664526,0.367097,0.258621,0.152913,0.441942,0.179921,0.209790
1,0.400538,0.302835,0.337879,0.474906,0.491189,0.680380,0.258222,0.284938,0.250000,0.000000,0.166667,1.000000,1.000000,0.272656,0.252302,0.602752,0.613854,0.514294,0.524883,0.010000,0.350649,0.218978,0.048387,0.258621,0.152913,0.135922,0.023715,0.209790
2,0.400538,0.302835,0.337879,0.474906,0.491189,0.642405,0.258222,0.284938,0.250000,0.000000,0.166667,1.000000,1.000000,0.272656,0.252302,0.602752,0.613854,0.514294,0.524883,0.026667,0.350649,0.218978,0.112903,0.258621,0.152913,0.135922,0.055336,0.209790
3,0.400538,0.302835,0.337879,0.474906,0.491189,0.683544,0.258222,0.284938,0.250000,0.000000,0.166667,1.000000,1.000000,0.272656,0.252302,0.602752,0.613854,0.514294,0.524883,0.023333,0.350649,0.218978,0.080645,0.258621,0.152913,0.135922,0.039526,0.209790
4,0.400538,0.302835,0.337879,0.474906,0.491189,0.664557,0.258222,0.284938,0.250000,0.000000,0.166667,1.000000,1.000000,0.272656,0.252302,0.602752,0.613854,0.514294,0.524883,0.023333,0.350649,0.218978,0.088710,0.258621,0.152913,0.135922,0.043478,0.209790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,0.004839,0.003222,0.351010,0.344944,0.612335,0.382911,0.316475,0.263755,0.500000,0.260870,0.666667,0.843776,0.813817,0.003940,0.000921,1.000000,0.987832,0.670511,0.653564,0.080000,0.591169,0.613139,0.278226,0.724138,0.193010,0.388350,0.106719,0.629371
2266582,0.004839,0.003222,0.276768,0.268914,0.603524,0.563291,0.244415,0.206111,0.666667,0.043478,0.666667,0.843776,0.813817,0.003152,0.003683,0.814711,0.823170,0.931764,0.916060,0.151733,0.591169,0.452555,0.367097,0.784828,0.220971,0.281553,0.179921,0.636644
2266583,0.004839,0.003222,0.276768,0.268914,0.603524,0.550633,0.244415,0.206111,0.666667,0.043478,0.666667,0.843776,0.813817,0.003152,0.003683,0.814711,0.823170,0.931764,0.916060,0.036667,0.591169,0.452555,0.137097,0.784828,0.220971,0.281553,0.063241,0.636644
2266584,0.004839,0.003222,0.276768,0.268914,0.603524,0.686709,0.244415,0.206111,0.666667,0.043478,0.666667,0.843776,0.813817,0.003152,0.003683,0.814711,0.823170,0.931764,0.916060,0.153333,0.591169,0.452555,0.112903,0.784828,0.220971,0.281553,0.055336,0.636644


In [43]:
y = original_data.copy()['answerCode']
y

0          1
1          1
2          1
3          1
4          1
          ..
2266581    0
2266582    1
2266583    1
2266584    1
2266585    1
Name: answerCode, Length: 2266586, dtype: int64

In [44]:
selector = SelectKBest(chi2, k=19) # 선택하고자 하는 변수의 개수를 k 값으로 지정해줄 수 있음
selector.fit_transform(X, y).shape

(2266586, 19)

In [45]:
selector.scores_

array([5.31131869e+02, 5.27666478e+03, 1.69177500e+03, 8.71293014e+02,
       6.72442990e+01, 1.73365789e+04, 9.93919164e+02, 4.64629947e+03,
       2.48084800e+02, 1.47594452e+01, 2.12441126e+02, 1.78841321e+01,
       2.71390362e+01, 8.16732261e+02, 1.28227078e+04, 4.32556340e+01,
       1.19780127e+02, 3.10830736e+02, 3.85138430e+02, 5.39689927e+03,
       6.17634213e+03, 1.45844703e+03, 8.74250447e+01, 1.76293716e+03,
       2.27119186e+04, 3.36109851e+04, 1.21370962e+04, 5.10092651e+04])

In [46]:
# 선택된 feature의 정보
X.columns[selector.get_support()]

Index(['userID_answerCode_count', 'userID_answerCode_sum',
       'testId_answerCode_count', 'testId_answerCode_sum',
       'assessmentItemID_answerCode_sum', 'KnowledgeTag_answerCode_count',
       'KnowledgeTag_answerCode_sum', 'userID_first3_answerCode_count',
       'userID_first3_answerCode_sum', 'month_answerCode_count',
       'month_answerCode_sum', 'elapsedTime', 'userID_elapsedTime_median',
       'KnowledgeTag_elapsedTime_median', 'testId_elapsedTime_median',
       'userID_answerCode_elapsedTime_median',
       'KnowledgeTag_answerCode_elapsedTime_median',
       'assessmentItemID_answerCode_elapsedTime_median',
       'testId_answerCode_elapsedTime_median'],
      dtype='object')

## 결론
* 분산 2를 기준으로 변별력이 낮은 변수는 제외
* 카이제곱 독립성 검정을 통해, 상관관계가 높은 변수들을 선별

* 선택된 변수
* 범주형: 'userID', 'assessmentID', 'testId', 'KnowledgeTag', 'first3', 'hour_answerCode_Level' / 'answerCode' (target)
* 연속형: 'userID_answerCode_count', 'userID_answerCode_sum',
       'testId_answerCode_count', 'testId_answerCode_sum',
       'assessmentItemID_answerCode_sum', 'KnowledgeTag_answerCode_count',
       'KnowledgeTag_answerCode_sum', 'userID_first3_answerCode_count',
       'userID_first3_answerCode_sum', 'month_answerCode_count',
       'month_answerCode_sum', 'elapsedTime', 'userID_elapsedTime_median',
       'KnowledgeTag_elapsedTime_median', 'testId_elapsedTime_median',
       'userID_answerCode_elapsedTime_median',
       'KnowledgeTag_answerCode_elapsedTime_median',
       'assessmentItemID_answerCode_elapsedTime_median',
       'testId_answerCode_elapsedTime_median'

* 사용하고자 하는 연속형 변수의 수를 줄이고 싶다면, 카이제곱 독립성 검정에 사용하는 k 값을 낮춰주면 됩니다